## SMIB

Synchronous Machine Infinite Bus (SMIB) system


### System building

In [8]:
import numpy as np
from pydae.bmapu import bmapu_builder

#### System data

In [9]:
zeta = 0.1 # 1.0/np.sqrt(2) 
H_v = 4.0 
WB = 2 *np.pi* 50;
R_v = 0.0
X_v = 0.3

Lt = X_v 
P_max = 1/Lt
fn = np.sqrt(WB*P_max/(2*H_v))/(2*np.pi)

Dp = 0;
K_p = 0.01
T_p = 0.01


T_q = 1.0/(2*np.pi*10/2)
K_q = (1.0 - 0.0)/(1.05 - 0.95)
K_i_q = 1e-6

theta_red = 3.0/180*np.pi
V = 1.0
p_ref = 0.9
q_ref = 0.434616
v_ref = 1.0
T_q = 1.0/(2*np.pi*10/2)

In [10]:
data = {
"system":{"name":"smib_vsc_l","S_base":100e6, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},       
"buses":[{"name":"1", "P_W":0.0,"Q_var":0.0,"U_kV":20.0},
         {"name":"2", "P_W":0.0,"Q_var":0.0,"U_kV":20.0}
        ],
"lines":[{"bus_j":"1", "bus_k":"2", "X_pu":0.05,"R_pu":0.01,"Bs_pu":1e-6,"S_mva":100.0}],
"vscs": [{"bus":"1","type":"vsc_lcl_uc","S_n":10e6,"F_n":50.0,"X_s":0.05,"R_s":0.005,"C_u":100.0,"R_u":0.01,
          "ctrl":{"type":"uvsg","K_delta":0.0,"F_n":50.0,
                  "R_v":R_v,"X_v":X_v,
                  "K_p":K_p,"T_p":T_p,"K_q":K_q,"T_q":T_q,"K_u":0.1,"T_u":100.0}}],
"sources":[
    {"type":"genape","bus":"2","S_n":100e6,"F_n":50.0,"X_v":0.001,"R_v":0.0,"K_delta":0.001,"K_alpha":1e-6}]
}

#### System generation and compilation

In [11]:
grid = bmapu_builder.bmapu(data)
#grid.checker()
grid.uz_jacs = True
grid.verbose = False
grid.build('vsc_lcl_uc_uvsg')

In [12]:
data_syn = {
"system":{"name":"smib","S_base":20e3, "K_p_agc":0.0,"K_i_agc":0.0,"K_xif":0.01},       
"buses":[{"name":"1", "P_W":0.0,"Q_var":0.0,"U_kV":20.0},
         {"name":"2", "P_W":0.0,"Q_var":0.0,"U_kV":20.0}
        ],
"lines":[{"bus_j":"1", "bus_k":"2", "X_pu":0.05,"R_pu":0.01,"Bs_pu":1e-6,"S_mva":100.0}],
"syns":[
      {"bus":"1","S_n":100e6,
         "X_d":1.8,"X1d":0.3, "T1d0":8.0,    
         "X_q":1.7,"X1q":0.55,"T1q0":0.4,  
         "R_a":0.01,"X_l": 0.2, 
         "H":5.0,"D":1.0,
         "Omega_b":314.1592653589793,"omega_s":1.0,"K_sec":0.0,
         "avr":{"type":"kundur_tgr","K_a":200,"T_r":0.01,"E_fmin":-5,"E_fmax":10.0,"T_a":1,"T_b":10,"v_ref":1.03},
         "pss":{"type":"pss_kundur_2","K_stab":20, "T_1":0.05, "T_2":0.02, "T_3":3.0, "T_4":5.4, "T_w":10.0},      
         "K_delta":0.0}],
"sources":[
    {"type":"genape","bus":"2","S_n":100e6,"F_n":50.0,"X_v":0.001,"R_v":0.0,"K_delta":0.001,"K_alpha":1e-6}]
}

In [6]:
grid = bmapu_builder.bmapu(data_syn)
#grid.checker()
grid.uz_jacs = True
grid.verbose = False
grid.build('smib_syn')